In [ ]:
import random
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from transformer_type.model import TransAm
from transformer_type.dataset import get_data
import torch

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

if torch.cuda.is_available(): device = torch.device("cuda")
elif torch.backends.mps.is_available(): device = torch.device("mps")
else: device = torch.device('cpu')

In [ ]:
# input_window, output_window, batch_size, epochs, feature_size, d_ff, num_layers, lr, dropout
params = [[120, 1, 32, 300, 128, 128, 2, 1e-3, 0.1],
          [120, 1, 32, 300, 128, 64, 1, 1e-3, 0.1],
          [120, 1, 32, 300, 128, 512, 2, 1e-3, 0.1],
          [120, 1, 32, 300, 128, 256, 2, 1e-3, 0.1],
          [120, 1, 32, 300, 256, 64, 1, 1e-3, 0.1]]

In [ ]:
MODEL_PATH = ""
type_list = ['A', 'B', 'C', 'D', 'E', 'G', 'H', 'I']

In [ ]:
train_data_list = []
val_data_list = []

for type_name in type_list:
    train_data, val_data, scaler = get_data(type_name, params[0], params[1])
    train_data = train_data.to(device)
    val_data = val_data.to(device)
    train_data_list.append((train_data, type_name, scaler))
    val_data_list.append((val_data, type_name, scaler))

In [ ]:
models = {t : TransAm(*params).load_state_dict(torch.load(MODEL_PATH + f"{t}_.pt")) for t in type_list}
cur_values = {}
predictions = {}
true_values = {}
input_window, output_window = params[:2]

for val_data, t, scaler in val_data_list:
    model = models[t]
    model.eval()
    cur_values[t] = {}
    predictions[t] = []
    true_values[t] = []
    for i in range(params[0], len(val_data) - 180):
        _, data, scaler = val_data[i]
        cur_values[t].append(data[-1])
        for _ in range(180):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
        predictions[t].append(output.item())
        true_values[t].append(val_data[i + 180][1][-1])   